In [1]:
import numpy as np
from pyscf import fci
from copy import deepcopy, copy

#define vacuum
#take in operator string
#format: string of indices, bitstring of creation vs annihilation
#return normal-ordered operator string plus contracted terms, according to Wick's theorem



In [2]:
class basicOperator:
    def __init__(self, orbital_, creation_annihilation_, spin_):
        self.orbital = orbital_ #orbital index
        self.spin = spin_ #1 for alpha, 0 for beta
        self.creation_annihilation = creation_annihilation_ #1 for creation, 0 for annihilation

    def apply(self, state):
        neleca, nelecb = state.nelec[0], state.nelec[1]
        if bool(self.creation_annihilation):
            if bool(self.spin):
                state.array = fci.addons.cre_a(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca + 1, nelecb)
            else:
                state.array = fci.addons.cre_b(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca, nelecb + 1)
        else:
            if bool(self.spin):
                state.array = fci.addons.des_a(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca - 1, nelecb)
            else:
                state.array = fci.addons.des_b(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca, nelecb - 1)

In [74]:
class operatorSum:
    def __init__(self, operatorList_):
        self.operatorList = operatorList_

    def apply(self, state):
        result = 0
        for o in self.operatorList:
            statecopy = copy(state)
            o.apply(statecopy)
            result = statecopy + result
#        state1, state2 = copy(state), copy(state)
#        self.operator1.apply(state1)
#        self.operator2.apply(state2)
#        state = state1 + state2
        state = result

In [72]:
class operatorProduct:
    def __init__(self, operatorList_):
        self.operatorList = operatorList_

    def apply(self, state):
        i = len(self.operatorList)
        while i>0:
            i = i - 1
            self.operatorList[i].apply(state)

In [73]:
#Normal-ordered particle number-conserving excitation operators
class excitation(operatorProduct):
    def __init__(self, creationList_, annihilationList_, spinList_):
        self.operatorList = []
        for i in range(len(creationList_)):
            self.operatorList.append(basicOperator(creationList_[i], 1, spinList_[i]))
        for i in range(len(annihilationList_)):
            self.operatorList.append(basicOperator(annihilationList_[i], 0, spinList_[i+len(creationList_)]))

In [77]:
class spinFreeExcitation(operatorSum):
    def __init__(self, creationList_, annihilationList_):
        self.operatorList = []
        spinLists = np.reshape(np.zeros(len(creationList_)), (1, -1))
        for i in range(len(creationList_)):
            newspinLists = spinLists.copy()
            for s in range(len(spinLists)):
                newspinLists[s,i] = 1
            spinLists = np.concatenate((spinLists, newspinLists))
        for l in range(len(spinLists)):
            spinList = spinLists[l]
            self.operatorList.append(excitation(creationList_, annihilationList_, spinList + spinList[::-1]))

In [4]:
class operatorString:
    def __init__(self, string_):
        self.string = string_
        #list of indices in the string of second-quantised operators
        self.iS = []
        #bitstring of same length, with 1 corresponding to creation and 0 for annihilation
        self.cOA = []
        #bitstring of same length, with 1 corresponding to alpha and 0 for beta
        self.aOB = []

    #Get indices, alpha or beta, and creation or annihilation from string
    def parse(self):
        c = 0
        while c < len(self.string) - 1:
            self.iS.append(int(self.string[c]))
            if self.string[c+1] == "a":
                    self.aOB.append(1)
            elif self.string[c+1] == "b":
                    self.aOB.append(0)
            if c < len(self.string) - 2 and self.string[c+2] == "+":
                self.cOA.append(1)
                c = c + 3
            else:
                self.cOA.append(0)
                c = c + 2

    #Change string based on new indices, alpha or beta, and creation or annihilation
    def updateString(self):
        self.string = ""
        i = 0
        while i < len(self.iS):
            if bool(self.cOA[i]):
                if bool(self.aOB[i]):
                    self.string = self.string + str(self.iS[i]) + "a+"
                else:
                    self.string = self.string + str(self.iS[i]) + "b+"
            else:
                if bool(self.aOB[i]):
                    self.string = self.string + str(self.iS[i]) + "a"
                else:
                    self.string = self.string + str(self.iS[i]) + "b"
            i = i + 1

    #Apply the string of second-quantised operators to an arbitrary state object
    #Note that nelec does not yet account for the state being completely annihilated
    def apply(self, state):
        i = len(self.iS)
        while(i > 0):
            i = i - 1
#            print(i)
#            print(self.iS[i])
#            print(self.cOA[i])
#            print(self.aOB[i])
            neleca, nelecb = state.nelec[0], state.nelec[1]
            if bool(self.cOA[i]):
                if bool(self.aOB[i]):
                    state.array = fci.addons.cre_a(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca + 1, nelecb)
                else:
                    state.array = fci.addons.cre_b(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca, nelecb + 1)
            else:
                if bool(self.aOB[i]):
                    state.array = fci.addons.des_a(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca - 1, nelecb)
                else:
                    state.array = fci.addons.des_b(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca, nelecb - 1)

In [82]:
class state:
    def __init__(self, array_, norb_, nelec_):
        self.array = array_
        self.norb = norb_
        self.nelec = nelec_

    def __add__(self, other):
        if other == 0:
            return self
        elif self.norb == other.norb and self.nelec == other.nelec:
            newArray = self.array + other.array
            return state(newArray, self.norb, self.nelec)
        else:
            print("Adding states of different orbital or particle numbers or different spin projections is not supported. Returning original state")
            return self

    def __radd__(self, other):
        if other == 0:
            return self
        else:
            return self.__add__(other)


In [84]:
ref = state(np.array([[1.,0.,0.],[0.,0.,0.],[0.,0.,0.]]), 3, (2,1))

In [85]:
opStr = operatorString("0a1b+2a+")
opStr.parse()

In [86]:
opStr.iS

[0, 1, 2]

In [87]:
new = deepcopy(ref)
opStr.apply(new)

2
2
1
1
1
1
1
0
0
0
0
1


In [88]:
ref.nelec

(2, 1)

In [89]:
new.nelec

(2, 2)

In [90]:
new.array

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-1.,  0.,  0.]])

In [91]:
opStr1 = operatorString("0a+0b+1a+")
opStr1.parse()
print(opStr1.string)
print(opStr1.iS)
print(opStr1.aOB)
print(opStr1.cOA)

0a+0b+1a+
[0, 0, 1]
[1, 0, 1]
[1, 1, 1]


In [92]:
opStr2 = copy(opStr1)

In [93]:
type(opStr2.string)

str

In [94]:
opStr2.iS[2] = 2
print(opStr2.string)
print(opStr2.iS)
print(opStr2.aOB)
print(opStr2.cOA)

0a+0b+1a+
[0, 0, 2]
[1, 0, 1]
[1, 1, 1]


In [95]:
opStr2.updateString()

In [96]:
opStr2.string

'0a+0b+2a+'

In [97]:
ref.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [98]:
new = copy(ref)

In [99]:
new.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [100]:
opStr.apply(new)

2
2
1
1
1
1
1
0
0
0
0
1


In [101]:
new.array

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-1.,  0.,  0.]])

In [102]:
cre0a = basicOperator(0,1,1)

In [103]:
cre0a.apply(new)

In [104]:
new.array

array([[-1.,  0.,  0.]])

In [105]:
new.nelec

(3, 2)

In [107]:
des1 = operatorSum([basicOperator(1,1,0), basicOperator(1,0,0)])

In [108]:
des1.apply(new)

Adding states of different orbital or particle numbers or different spin projections is not supported. Returning original state


In [109]:
E12 = spinFreeExcitation([2],[1])

In [110]:
E12.apply(new)

Adding states of different orbital or particle numbers or different spin projections is not supported. Returning original state
